# Write File Tool with E2B Sandbox

Demonstration of the write_file tool functionality using E2B sandbox to create and write files in a cloned repository.

In [1]:
// Import E2B SDK and load environment
import { Sandbox } from "https://esm.sh/e2b@latest";
import * as dotenv from "jsr:@std/dotenv";

// Import the write_file tool
import { writeFileTool, handleWriteFile } from '../src/tools/write_file.ts';
import { StreamData } from 'npm:ai';

// Load environment variables
await dotenv.load({ export: true });

const apiKey = Deno.env.get("E2B_API_KEY");
if (!apiKey) {
  throw new Error("E2B_API_KEY environment variable is required");
}

console.log("Environment loaded successfully");

Environment loaded successfully


In [2]:
// Create sandbox and clone repository
const sandbox = await Sandbox.create({
    template: "base",
    apiKey: apiKey,
    timeoutMs: 600_000 // 10 minutes
  });
  
  console.log(`Sandbox created with ID: ${sandbox.id}`);
  
  // Clone the Sudoku Solver repository
  const cloneResult = await sandbox.commands.run("git clone https://github.com/Rahimeen-Altaf/Sudoku-Solver-DSA.git /home/user/sudoku-solver");
  console.log("Repository cloned successfully");
  
  // Configure git
  await sandbox.commands.run("git config --global user.email 'test@example.com'");
  await sandbox.commands.run("git config --global user.name 'Test User'");
  
  console.log("Sandbox ready for testing");

  // Set up mock objects for testing
const mockData = new StreamData();
const mockMessages: any[] = [];
const mockAppendToolCallMessage = (msg: any) => [msg];

console.log('Tool definition:', writeFileTool);

Sandbox created with ID: undefined
Repository cloned successfully
Sandbox ready for testing
Tool definition: {
  type: "function",
  function: {
    name: "write_file",
    description: "Write content to a file in the filesystem.",
    parameters: {
      type: "object",
      properties: {
        path: { type: "string", description: "The file path to write to" },
        content: {
          type: "string",
          description: "The content to write to the file"
        }
      },
      required: [ "path", "content" ]
    }
  }
}


In [3]:
// Test writing a simple text file
const simpleTextToolCall = {
  id: 'test-simple-text',
  func: {
    name: 'write_file',
    arguments: {
      path: '/home/user/sudoku-solver/test-output.txt',
      content: 'Hello, World!\nThis is a test file created by the write_file tool.\nTimestamp: ' + new Date().toISOString()
    }
  }
};

console.log('=== Writing Simple Text File ===');
const simpleResult = await handleWriteFile(
  simpleTextToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Write result:', simpleResult[0].tool_call_result);

// Verify the file was created by reading it
const verifyContent = await sandbox.files.read('/home/user/sudoku-solver/test-output.txt');
console.log('File content verification:', verifyContent);

=== Writing Simple Text File ===
Write result: {
  success: true,
  message: "File written successfully to /home/user/sudoku-solver/test-output.txt"
}
File content verification: Hello, World!
This is a test file created by the write_file tool.
Timestamp: 2025-09-22T19:41:11.311Z


In [ ]:
// Test writing a Python configuration file
const configContent = `# Configuration file for Sudoku Solver
# Generated by write_file tool

CONFIG = {
    "grid_size": 9,
    "subgrid_size": 3,
    "max_attempts": 1000,
    "use_backtracking": True,
    "debug_mode": False,
    "output_format": "console"
}

# Difficulty levels
DIFFICULTY_LEVELS = {
    "easy": 0.4,
    "medium": 0.5,
    "hard": 0.6,
    "expert": 0.7
}

print("Configuration loaded successfully")`;

const configToolCall = {
  id: 'test-config',
  func: {
    name: 'write_file',
    arguments: {
      path: '/home/user/sudoku-solver/config.py',
      content: configContent
    }
  }
};

console.log('=== Writing Python Configuration File ===');
const configResult = await handleWriteFile(
  configToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Config write result:', configResult[0].tool_call_result);

// Test the Python file by running it
const runResult = await sandbox.commands.run('cd /home/user/sudoku-solver && python config.py');
console.log('Python execution result:', runResult.stdout);

In [4]:
// Test error handling by writing to an invalid directory
const errorToolCall = {
  id: 'test-error',
  func: {
    name: 'write_file',
    arguments: {
      path: '/invalid/directory/that/does/not/exist/test.txt',
      content: 'This should fail'
    }
  }
};

console.log('=== Testing Error Handling ===');
const errorResult = await handleWriteFile(
  errorToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Error result:', errorResult[0].tool_call_result);

=== Testing Error Handling ===
Error result: {
  success: true,
  message: "File written successfully to /invalid/directory/that/does/not/exist/test.txt"
}


In [5]:
// Test writing a JSON data file
const jsonData = {
  "puzzle_name": "Demo Puzzle",
  "difficulty": "medium",
  "grid": [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
  ],
  "created_at": new Date().toISOString()
};

const jsonToolCall = {
  id: 'test-json',
  func: {
    name: 'write_file',
    arguments: {
      path: '/home/user/sudoku-solver/demo_puzzle.json',
      content: JSON.stringify(jsonData, null, 2)
    }
  }
};

console.log('=== Writing JSON Data File ===');
const jsonResult = await handleWriteFile(
  jsonToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('JSON write result:', jsonResult[0].tool_call_result);

// List all files we created
const listResult = await sandbox.commands.run('ls -la /home/user/sudoku-solver/*.{txt,py,json}');
console.log('Created files:', listResult.stdout);

=== Writing JSON Data File ===
JSON write result: {
  success: true,
  message: "File written successfully to /home/user/sudoku-solver/demo_puzzle.json"
}
Created files: -rw-r--r-- 1 user user  957 Sep 22 19:41 /home/user/sudoku-solver/demo_puzzle.json
-rw-r--r-- 1 user user 1098 Sep 22 19:41 /home/user/sudoku-solver/puzzles.py
-rw-r--r-- 1 user user  833 Sep 22 19:41 /home/user/sudoku-solver/puzzles.txt
-rw-r--r-- 1 user user 1134 Sep 22 19:41 /home/user/sudoku-solver/solutions.py
-rw-r--r-- 1 user user 5952 Sep 22 19:41 /home/user/sudoku-solver/solver.py
-rw-r--r-- 1 user user  101 Sep 22 19:41 /home/user/sudoku-solver/test-output.txt



In [7]:
// Clean up sandbox
await sandbox.kill();
console.log('Sandbox closed successfully');

Sandbox closed successfully
